In [40]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import linregress
from scipy import odr
from scipy.signal import butter,filtfilt, find_peaks

In [41]:
###### TO CHANGE FOR THE ZOOM IN TO EACH CYCLE

# First: n = 660 & m = 710
# Second: n = 1200 & m = 1260
# Third: n = 3980 & m = 4030
# Fourth: n = 4510 & m = 4560
# Fifth: n = 5200 & m = 6500

## Limit of sample step
n = 5025
m = 5075


## Limit for stress ranges
# lim1 = 
# lim2 = 100

disp = 0.01

In [42]:
df_exp_PB = pd.read_csv('Correction_70-30_122023.csv')                                                            # Read the file
df_exp_PB_1 = df_exp_PB.iloc[n:m]

stress1 = df_exp_PB_1['Stress [MPa]']             # Apply filter to stress
force = df_exp_PB_1['Force(kN)']
strain1 = df_exp_PB_1['Strain Ax [-]']            # Apply filter to strain
strain_al = df_exp_PB_1['Strain Al. [-]']
strain_mach = df_exp_PB_1['Strain Machine [-]']


## DIFFERENT WAY OF READING THE DATA, SOMETIMES IT HELPS WITH DIFFERENT FUNCTIONS

stress2 = stress1.to_numpy()
strain2 = strain1.to_numpy()

# print(stress2)

LVDT1_1 = df_exp_PB['LVDT1']
LVDT2_2 = df_exp_PB['LVDT2']
LVDT1 = LVDT1_1.to_numpy()
LVDT2 = LVDT2_2.to_numpy()
LVDT1_min = LVDT1.min()
LVDT2_min = LVDT2.min()
sample_length = 70
LVDT1_norm = np.array(())
LVDT2_norm = np.array(())
LVDT_avg = np.array(())
Strain_LVDT = np.array(())

for value in range(len(LVDT1)):
    LVDT1_norm = np.append(LVDT1_norm, LVDT1[value] - LVDT1_min)
    LVDT2_norm = np.append(LVDT2_norm, LVDT2[value] - LVDT2_min)

    LVDT_avg = np.append(LVDT_avg, (LVDT1_norm[value] + LVDT2_norm[value])/2)

    Strain_LVDT = np.append(Strain_LVDT, LVDT_avg[value]/sample_length)


print(Strain_LVDT)

[3.13214286e-05 1.72357143e-05 2.20571429e-05 ... 3.55145000e-03
 3.55318571e-03 3.55168571e-03]


In [ ]:
### Plot specific cycle
fig = go.Figure()
fig.add_trace(go.Scatter(x=strain1, y=stress1))

fig.update_layout(template="simple_white", showlegend=False, autosize=False, width=700, height=700, 
    title="First cycle",
    xaxis_title="Strain Ax [-]",
    yaxis_title="Stress [MPa]",
    font=dict(
        family="New Times Roman",
        size=20,
        color="Black"))

fig.update_yaxes(showgrid=True)
fig.update_xaxes(showgrid=True)

fig.show()

In [ ]:
### Different strains
fig = go.Figure()
# fig.add_trace(go.Scatter(x=strain1, y=stress1))
# fig.add_trace(go.Scatter(x=strain_al, y=stress1))
# fig.add_trace(go.Scatter(x=strain_mach, y=stress1))

# fig.add_trace(go.Scatter(x=strain1, y=force))
# fig.add_trace(go.Scatter(x=strain_al, y=force))
# fig.add_trace(go.Scatter(x=strain_mach, y=force))

fig.add_trace(go.Scatter(name='Total Strain', x=stress1, y=strain1))
fig.add_trace(go.Scatter(name='Alumninium Strain', x=stress1, y=strain_al))
fig.add_trace(go.Scatter(name='Machine Strain', x=stress1, y=strain_mach))

# fig.add_trace(go.Scatter(x=force, y=strain1))
# fig.add_trace(go.Scatter(x=force, y=strain_al))
# fig.add_trace(go.Scatter(x=force, y=strain_mach))


fig.update_layout(template="simple_white", showlegend=True, autosize=False, width=700, height=700, 
    # title="First cycle",
    xaxis_title="Strain [-]",
    # yaxis_title="Force [kN]",
    yaxis_title="Stress [MPa]",
    font=dict(
        family="New Times Roman",
        size=20,
        color="Black"),
    legend=dict(title=None, orientation='h', y=1, yanchor="bottom", x=1, xanchor="right", font=dict(size = 16), bordercolor='black', borderwidth=1))

fig.update_yaxes(showgrid=True)
fig.update_xaxes(showgrid=True)

fig.show()

In [ ]:
z = np.polyfit(strain_mach, stress1,1) # put here your x and y of the data which you want to fit, the last argument is the order of the polyfit
p = np.poly1d(z) # returns 1d polynomial object 
print(p)
# look at this webpage:
#https://numpy.org/doc/stable/reference/generated/numpy.poly1d.html#numpy.poly1d

##HOW WELL DOES IT FIT
corr_matrix = np.corrcoef(strain1, stress1)
corr = corr_matrix[0,1]
R_sq = corr**2
print(R_sq)

In [ ]:
import matplotlib.pyplot as plt
xp = np.linspace(np.min(strain_mach), np.max(strain_mach), 100) # create linspace, so you can then take values from your fitted function
plt.figure(figsize=(10,10))
plt.plot(strain_mach, stress1, '.', xp, p(xp), '-') # plot you original function vs the fitterd one
plt.show()

In [49]:
### PLOTTING
## Plot the whole data
 
fig = go.Figure()
fig.add_trace(go.Scatter(name='Total Strain1 from Computer', x=df_exp_PB['Strain Ax [-]'], y=df_exp_PB['Stress [MPa]'], marker=dict(color='DarkBlue')))
fig.add_trace(go.Scatter(name='Strain Machine from Total Strain1', x=df_exp_PB['Strain Machine [-]'], y=df_exp_PB['Stress [MPa]'], marker=dict(color='Blue')))
fig.add_trace(go.Scatter(name='Strain Aluminium', x=df_exp_PB['Strain Al. [-]'], y=df_exp_PB['Stress [MPa]'], marker=dict(color='LightBlue')))


## Calculated strain with LVDTS
fig.add_trace(go.Scatter(name='Total Strain2 from LVDTs', x=df_exp_PB['Total Strain (-)'], y=df_exp_PB['Stress [MPa]'], marker=dict(color='DarkGreen')))
fig.add_trace(go.Scatter(name='Strain Machine from Total Strain2', x=df_exp_PB['Strain Machine'], y=df_exp_PB['Stress [MPa]'], marker=dict(color='Green')))
fig.add_trace(go.Scatter(name='Strain Aluminium', x=df_exp_PB['Strain Alum'], y=df_exp_PB['Stress [MPa]'], marker=dict(color='LightGreen')))


fig.update_layout(template="simple_white", showlegend=True, autosize=False, width=700, height=700, 
    # title="First cycle",
    xaxis_title="Strain Ax [-]",
    yaxis_title="Stress [MPa]",
    font=dict(
        family="New Times Roman",
        size=20,
        color="Black"),
    legend=dict(title=None, orientation='h', y=1, yanchor="bottom", x=1, xanchor="right", font=dict(size = 16), bordercolor='black', borderwidth=1))

fig.update_yaxes(showgrid=True)
fig.update_xaxes(showgrid=True)

fig.show()